# 

분류성능평가지표 - Precision(정밀도), Recall(재현율) and Accuracy(정확도)
https://sumniya.tistory.com/26

In [4]:
recall = 86
prec = 91.44
f1 = (2*recall*prec)/(recall+prec)
print(f1)

88.6366095581605


In [5]:
(99.5+91+56+76.23)/4


80.6825

In [ ]:
# label 교체
new_test_label_ = torch.where(test_label_==3,torch.tensor([2]),test_label_)

In [ ]:
def convertclass(origin_label, from_classes=[],to_class=[]):
    converted_label = origin_label
    for from_class in from_classes:
        converted_label = torch.where(converted_label==from_class, torch.tensor([to_class[0]]), converted_label)# label 교체
    return converted_label



# Main Eval Calss

In [31]:
from yjlib.ShowResult import ShowResult
import matplotlib.pyplot as plt
%matplotlib inline
from prettytable import PrettyTable

class Evaluation():
    """Evaluation model
    
    init
    ----
    result dictionary
        train loss and acc
        test loss, acc, and classes add
    
    Methods
    -------
    train:
    test:
    reset_result:
    save_result:
    show_loss:
    show_acc:
    show_class_acc:
    show_confusion_matrix:
    show_roc:
    show_heatmap:
    
    
    """
    def __init__(self, NAME_OF_CLASS, device):
        self.result = dict()
        self.result["train_losses"] = []
        self.result["train_accuracyes"] = []
        
        self.result["test_losses"] = []
        self.result["test_accuracyes"] = []
        self.result["test_class_acc"] = []
        
        
        self.result["test_label"] = []
        self.result["test_y_pred"] = []
        
        self.device = device
        self.showresult = ShowResult()
        self.NAME_OF_CLASS = NAME_OF_CLASS
        
    # import tqdm
    def train(self, model, optimizer,loss_func, data_loader, parallel_device_ids=[]):
        """Train for classification

        Parameter
        ---------
        model: model
        optimizer: optimizer
        loss_func: loss_function
        data_loader: data_loader
        parallel_device_ids: [] Parallel device ids

        """

        if len(parallel_device_ids) > 0 and torch.cuda.device_count() >= len(parallel_device_ids): 
            model = torch.nn.DataParallel(model)
        elif torch.cuda.device_count() < len(parallel_device_ids): 
            print("ERROR: cuda.device < len(device_ids)")

        model.train()
        model.to(self.device)

        correct = 0

        for batch_idx, (data, label) in enumerate(data_loader):
            data, label = data.to(self.device), label.to(self.device)
            data, label = Variable(data), Variable(label.long())

            # zero the parameter gradients
            optimizer.zero_grad() 

            # forward + backward + optimize
            output = model(data)
    #         _, output_index = torch.max(output,1)# output: [0,1,0,...], output_index: 1
            output = output.type(torch.FloatTensor)
            label = label.type(torch.LongTensor)
            label = label.view(-1)

            loss = loss_func(output, label)
            """
            Input: (N,C) where C = number of classes
            Target: (N) where each value is 0 <= targets[i] <= C-1
            Output: scalar. If reduce is False, then (N) instead.
            """
            loss.backward()
            optimizer.step()
            pred = output.max(1)[1] # get the index of the max log-probability
            correct += list(torch.eq(pred,label)).count(1)

        self.result["train_losses"].append(loss.item())
        self.result["train_accuracyes"].append(100. * correct / len(data_loader.dataset))
        print('Train set: Loss: {:.6f}  Accuracy: {}/{} Total_ACC: ({:.2f}%)'.format(
            loss.item(),
            correct,
            len(data_loader)*data_loader.batch_size,
            100. * correct / len(data_loader)*data_loader.batch_size))
    
    # 2 classes
    def test(self, model, loss_func, test_loader):
        """Classification test

        Parameter
        ---------


        """
        model.eval()
        model.to(self.device)
        test_loss = 0
        correct = 0

        total_sample_counter = [0]* len(self.NAME_OF_CLASS)# 각 클래스별 sample 수를 counting
        classes_correct_samples = [0]* len(self.NAME_OF_CLASS) # 각 클래별 맞는 sample 수

        self.result["test_label"] = torch.Tensor([]).type(torch.LongTensor) # dtype: torch.int64
        self.result["test_y_pred"] = torch.Tensor([]).type(torch.LongTensor) # dtype: torch.int64
        
        with torch.no_grad():
            for idx,(data, label) in enumerate(test_loader):
                data, label = data.to(self.device), label.to(self.device)
                data, label = Variable(data), Variable(label.long())
                output = model(data)
                output = output.type(torch.FloatTensor)

                label = label.type(torch.LongTensor)
                label = label.view(-1) # dtype: torch.int64

                test_loss += loss_func(output, label).item() # sum up batch loss

                pred = output.max(1)[1] # get the index of the max log-probability # dtype: torch.int64

                # 각 클래스별 데이터 총량 계산
                for i in range(len(self.NAME_OF_CLASS)):
                    total_sample_counter[i] += list(label).count(i)

                # 맞는 전체 데이터 수 count
                correct += list(torch.eq(pred,label)).count(1)
                
                self.result["test_label"] = torch.cat((self.result["test_label"], label), 0)
                self.result["test_y_pred"] = torch.cat((self.result["test_y_pred"], pred), 0)

                # 각 클래스별로 맞는 갯수
                correct_indice = torch.eq(pred, label).nonzero().view(-1)
                for i in range(len(self.NAME_OF_CLASS)):
                    classes_correct_samples[i] += list(label[correct_indice]).count(i)

        
        
        acc_dict = dict()
        show_contents = []
        # counting number of correct for each classes
        for idx, name in enumerate(self.NAME_OF_CLASS):
            acc_dict[name] = classes_correct_samples[idx]/total_sample_counter[idx]
            show_contents.append(float("{0:.2f}".format(classes_correct_samples[idx]/total_sample_counter[idx])))

            
        # evaluation number
        acc_dict['average_acc'] = statistics.mean(acc_dict.values())
#         acc_dict['precision'] =  # precision 삽입
#         acc_dict['F-maeasure'] = # Multi class F-maeasure 삽입 예정 

        self.result["test_class_acc"].append(acc_dict)

        test_loss /= len(test_loader.dataset)

        self.result["test_losses"].append(test_loss)
        self.result["test_accuracyes"].append(100. * acc_dict["average_acc"])

        show_title =  self.NAME_OF_CLASS + ['Average loss','Average Accuracy']
        show_contents += [float("{0:.6f}".format(test_loss)),
                              float("{0:.2f}".format(100. * acc_dict["average_acc"]))]
        self.__print_result(show_title,show_contents)
                                 
        del show_contents, show_title
        
    def __print_result(self, title=[],contents=[]):
        print("Test set")
        assert len(title) == len(contents)
                                 
        # 클래스 별로 결과값 별로 pretty table 을 show
        info_table = PrettyTable(title)
        info_table.add_row(contents)
        
        print(info_table)

    def reset_result(self):
        """Reset result
        reset the result dictionary
        """
        print("reset result dictionary")
        self.result = dict()
        self.result["train_losses"] = []
        self.result["train_accuracyes"] = []
        
        self.result["test_losses"] = []
        self.result["test_accuracyes"] = []
        self.result["test_class_acc"] = []

        self.result["test_label"] = torch.Tensor([]).type(torch.LongTensor)
        self.result["test_y_pred"] = torch.Tensor([]).type(torch.LongTensor)
        
    def save_result(self, path):
        """Save result of acc and class acc
        Parameter
        --------
        path: absolute path for saving
        """
        print("saving result to csv")
        
    def show_loss(self):
        print("show loss")
        #loss
        data_list = [[x / 100 for x in self.result['train_losses']],
                     [x / 100 for x in self.result['test_losses']]]
        title = 'Loss Plot'
        ylabel = 'Loss'  
        tag = ['train','test']
        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.show()
        
    def show_acc(self):
        print("show acc")
        data_list = [[x / 100 for x in self.result['train_accuracyes']],
                     [x / 100 for x in self.result['test_accuracyes']]]
        title = 'Accuracy Plot'
        ylabel = 'Accuracy'   
        tag = ['train','test']
        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        ylim = [0,1]
        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.ylim(ylim)
        plt.show()
        
    def show_class_acc(self):
        print("show class acc")
        
        class_acc = dict()
        for name in self.NAME_OF_CLASS:
            class_acc[name] = []
        
        class_acc_result = []
        
        for dic in self.result["test_class_acc"]:
            # dictionary for each epoch
            for name in self.NAME_OF_CLASS:
                class_acc[name].append(dic[name])

        # class accuracy
        data_list = []
        for name in self.NAME_OF_CLASS:
            data_list.append(class_acc[name])
            
        title='Classes Accuracy'
        ylabel='Accuracy'
        # tag = ['Normal','lgd','hgd','cancer']a
        tag = self.NAME_OF_CLASS

        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        ylim = [0,1]
        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.ylim(ylim)
        plt.show()
        
    def show_confusion_matrix(self):
        print("show confusion matrix")
        self.cm = self.showresult.plot_confusion_matrix(self.NAME_OF_CLASS,
                                 label=self.result["test_label"], 
                                 predic=self.result["test_y_pred"])
        
    def show_roc(self):
        print("roc result")
        print(self.cm.T)
#         cm = self.cm.T
        
        
    
    def show_heatmap(self, data_path):
        """
        
        
        """
        print("show heatmap")
    def count_time(self):
        """
        Calculate time
        
        """
        start_t = datetime.datetime.now()
        

## Double label Evaluation

In [ ]:
from yjlib.ShowResult import ShowResult
import matplotlib.pyplot as plt
%matplotlib inline
from prettytable import PrettyTable

class Evaluation():
    """Evaluation model
    
    init
    ----
    result dictionary
        train loss and acc
        test loss, acc, and classes add
    
    Methods
    -------
    train:
    test:
    reset_result:
    save_result:
    show_loss:
    show_acc:
    show_class_acc:
    show_confusion_matrix:
    show_roc:
    show_heatmap:
    
    
    """
    def __init__(self, NAME_OF_CLASS, device):
        self.result = dict()
        self.result["train_losses"] = []
        self.result["train_accuracyes"] = []
        
        self.result["test_losses"] = []
        self.result["test_accuracyes"] = []
        self.result["test_class_acc"] = []
        
        self.result["test_label"] = []
        self.result["test_y_pred"] = []
        
        self.result['match_path'] = []
        self.result['match_label'] = torch.Tensor([]).type(torch.LongTensor)

        self.result['unmatch_path'] = []
        self.result['unmatch_label'] = torch.Tensor([]).type(torch.LongTensor)

        self.device = device
        self.showresult = ShowResult()
        self.NAME_OF_CLASS = NAME_OF_CLASS
        
    # import tqdm
    def train(self, model, optimizer,loss_func, data_loader, parallel_device_ids=[]):
        """Train for classification

        Parameter
        ---------
        model: model
        optimizer: optimizer
        loss_func: loss_function
        data_loader: data_loader
        parallel_device_ids: [] Parallel device ids

        """

        if len(parallel_device_ids) > 0 and torch.cuda.device_count() >= len(parallel_device_ids): 
            model = torch.nn.DataParallel(model)
        elif torch.cuda.device_count() < len(parallel_device_ids): 
            print("ERROR: cuda.device < len(device_ids)")

        model.train()
        model.to(self.device)

        correct = 0

        for batch_idx, (data, label) in enumerate(data_loader):
            data, label = data.to(self.device), label.to(self.device)
            data, label = Variable(data), Variable(label.long())

            # zero the parameter gradients
            optimizer.zero_grad() 

            # forward + backward + optimize
            output = model(data)
    #         _, output_index = torch.max(output,1)# output: [0,1,0,...], output_index: 1
            output = output.type(torch.FloatTensor)
            label = label.type(torch.LongTensor)
            label = label.view(-1)

            loss = loss_func(output, label)
            """
            Input: (N,C) where C = number of classes
            Target: (N) where each value is 0 <= targets[i] <= C-1
            Output: scalar. If reduce is False, then (N) instead.
            """
            loss.backward()
            optimizer.step()
            pred = output.max(1)[1] # get the index of the max log-probability
            correct += list(torch.eq(pred,label)).count(1)

        self.result["train_losses"].append(loss.item())
        self.result["train_accuracyes"].append(100. * correct / len(data_loader.dataset))
        print('Train set: Loss: {:.6f}  Accuracy: {}/{} Total_ACC: ({:.2f}%)'.format(
            loss.item(),
            correct,
            len(data_loader)*data_loader.batch_size,
            100. * correct / len(data_loader)*data_loader.batch_size))
    
    # 2 classes
    def test(self, model, loss_func, test_loader):
        """Classification test

        Parameter
        ---------


        """
        model.eval()
        model.to(self.device)
        test_loss = 0
        correct = 0

        total_sample_counter = [0]* len(self.NAME_OF_CLASS)# 각 클래스별 sample 수를 counting
        classes_correct_samples = [0]* len(self.NAME_OF_CLASS) # 각 클래별 맞는 sample 수

        self.result["test_label"] = torch.Tensor([]).type(torch.LongTensor) # dtype: torch.int64
        self.result["test_y_pred"] = torch.Tensor([]).type(torch.LongTensor) # dtype: torch.int64
        
        with torch.no_grad():
            for idx,(data, label, pathes) in enumerate(test_loader):
                data, label = data.to(self.device), label.to(self.device)
                data, label = Variable(data), Variable(label.long())
                output = model(data)
                output = output.type(torch.FloatTensor)

                label = label.type(torch.LongTensor)
                label = label.view(-1) # dtype: torch.int64
                
                # XXXX 표준화 필요
                new_label = self.__convertclass(label, from_classes=[3], to_class=[2])
                # XXXX 표준화 필요

                test_loss += loss_func(output, new_label).item() # sum up batch loss

                pred = output.max(1)[1] # get the index of the max log-probability # dtype: torch.int64

                # 각 클래스별 데이터 총량 계산
                for i in range(len(self.NAME_OF_CLASS)):
                    total_sample_counter[i] += list(label).count(i)

                # 맞는 전체 데이터 수 count
                correct += list(torch.eq(pred,label)).count(1)
                
                torch.eq(pred,label)
                
                # torch.eq(pred, label) 결과로 index 뽑기
                match_indic = torch.eq(pred,new_label)

                # path 및 label 저장 하기 
                self.result['match_path'] += [path for match, path in zip(match_indic,pathes) if match==True]
                self.result['match_label'] = torch.cat((self.result['match_label'],label[match_indic==True])) 

                self.result['unmatch_path'] += [path for match, path in zip(match_indic,pathes) if match==False]
                self.result['unmatch_label'] = torch.cat((self.result['unmatch_label'],label[match_indic==False])) 
                
                self.result["test_label"] = torch.cat((self.result["test_label"], new_label), 0)
                self.result["test_y_pred"] = torch.cat((self.result["test_y_pred"], pred), 0)

                # 각 클래스별로 맞는 갯수
                correct_indice = torch.eq(pred, label).nonzero().view(-1)
                for i in range(len(self.NAME_OF_CLASS)):
                    classes_correct_samples[i] += list(label[correct_indice]).count(i)

        
        
        acc_dict = dict()
        show_contents = []
        # counting number of correct for each classes
        for idx, name in enumerate(self.NAME_OF_CLASS):
            acc_dict[name] = classes_correct_samples[idx]/total_sample_counter[idx]
            show_contents.append(float("{0:.2f}".format(classes_correct_samples[idx]/total_sample_counter[idx])))

            
        # evaluation number
        acc_dict['average_acc'] = statistics.mean(acc_dict.values())
#         acc_dict['precision'] =  # precision 삽입
#         acc_dict['F-maeasure'] = # Multi class F-maeasure 삽입 예정 

        self.result["test_class_acc"].append(acc_dict)

        test_loss /= len(test_loader.dataset)

        self.result["test_losses"].append(test_loss)
        self.result["test_accuracyes"].append(100. * acc_dict["average_acc"])

        show_title =  self.NAME_OF_CLASS + ['Average loss','Average Accuracy']
        show_contents += [float("{0:.6f}".format(test_loss)),
                              float("{0:.2f}".format(100. * acc_dict["average_acc"]))]
        self.__print_result(show_title,show_contents)
                                 
        del show_contents, show_title
    
    def __convertclass(self, origin_label, from_classes=[],to_class=[]):
        converted_label = origin_label
        for from_class in from_classes:
            converted_label = torch.where(converted_label==from_class, torch.tensor([to_class[0]]), converted_label)# label 교체
        return converted_label

    def __print_result(self, title=[],contents=[]):
        print("Test set")
        assert len(title) == len(contents)
                                 
        # 클래스 별로 결과값 별로 pretty table 을 show
        info_table = PrettyTable(title)
        info_table.add_row(contents)
        
        print(info_table)

    def reset_result(self):
        """Reset result
        reset the result dictionary
        """
        print("reset result dictionary")
        self.result = dict()
        self.result["train_losses"] = []
        self.result["train_accuracyes"] = []
        
        self.result["test_losses"] = []
        self.result["test_accuracyes"] = []
        self.result["test_class_acc"] = []

        self.result["test_label"] = torch.Tensor([]).type(torch.LongTensor)
        self.result["test_y_pred"] = torch.Tensor([]).type(torch.LongTensor)
        
        self.result['match_path'] = []
        self.result['match_label'] = torch.Tensor([]).type(torch.LongTensor)

        self.result['unmatch_path'] = []
        self.result['unmatch_label'] = torch.Tensor([]).type(torch.LongTensor)
        
    def save_result(self, path):
        """Save result of acc and class acc
        Parameter
        --------
        path: absolute path for saving
        """
        print("saving result to csv")
        
    def show_loss(self):
        print("show loss")
        #loss
        data_list = [[x / 100 for x in self.result['train_losses']],
                     [x / 100 for x in self.result['test_losses']]]
        title = 'Loss Plot'
        ylabel = 'Loss'  
        tag = ['train','test']
        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.show()
        
    def show_acc(self):
        print("show acc")
        data_list = [[x / 100 for x in self.result['train_accuracyes']],
                     [x / 100 for x in self.result['test_accuracyes']]]
        title = 'Accuracy Plot'
        ylabel = 'Accuracy'   
        tag = ['train','test']
        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        ylim = [0,1]
        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.ylim(ylim)
        plt.show()
        
    def show_class_acc(self):
        print("show class acc")
        
        class_acc = dict()
        for name in self.NAME_OF_CLASS:
            class_acc[name] = []
        
        class_acc_result = []
        
        for dic in self.result["test_class_acc"]:
            # dictionary for each epoch
            for name in self.NAME_OF_CLASS:
                class_acc[name].append(dic[name])

        # class accuracy
        data_list = []
        for name in self.NAME_OF_CLASS:
            data_list.append(class_acc[name])
            
        title='Classes Accuracy'
        ylabel='Accuracy'
        # tag = ['Normal','lgd','hgd','cancer']a
        tag = self.NAME_OF_CLASS

        loc=  'best'
        xlabel = 'epoch'

        for i in range(len(data_list)):
            plt.plot(data_list[i])

        ylim = [0,1]
        plt.legend(tag,loc=loc)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xlabel(xlabel)
        plt.ylim(ylim)
        plt.show()
        
    def show_confusion_matrix(self):
        print("show confusion matrix")
        self.cm = self.showresult.plot_confusion_matrix(self.NAME_OF_CLASS,
                                 label=self.result["test_label"], 
                                 predic=self.result["test_y_pred"])
        
    def show_roc(self):
        print("roc result")
        print(self.cm.T)
#         cm = self.cm.T
        
        
    
    def show_heatmap(self, data_path):
        """
        
        
        """
        print("show heatmap")
    def count_time(self):
        """
        Calculate time
        
        """
        start_t = datetime.datetime.now()
        

        